In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import torch

from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from pathlib import Path

In [3]:
from ignite.metrics import Accuracy, Loss
from ignite.contrib.handlers import ProgressBar

In [4]:
from surname.dataset import ProjectDataset
from surname.mlp_model import Classifier
from surname.trainer import Trainer
from surname.args import args

In [5]:
path = Path('../data/surnames')
scratch = path/'scratch'
surnames_csv = path/args.proc_dataset_csv
vectorizer_path = scratch/args.vectorizer_fname
cw_file = scratch/args.cw_file
args.save_dir = scratch/args.dir_name
args.num_epochs = 2

args

Namespace(batch_size=64, checkpointer_name='classifier', checkpointer_prefix='surname', cw_file='class_weights.pt', device='cuda:3', dir_name='models', early_stopping_criteria=5, hidden_dim=300, learning_rate=0.001, num_epochs=2, proc_dataset_csv='surnames_with_splits.csv', raw_dataset_csv='surnames.csv', save_dir=PosixPath('../data/surnames/scratch/models'), save_every=2, save_total=5, train_proportion=0.7, vectorizer_fname='vectorizer.json')

In [6]:
df = pd.read_csv(surnames_csv)
df.head()

,nationality,nationality_index,split,surname
0,Arabic,15,train,Totah
1,Arabic,15,train,Abboud
2,Arabic,15,train,Fakhoury
3,Arabic,15,train,Srour
4,Arabic,15,train,Sayegh


In [7]:
is_load = True

In [8]:
if not is_load:
  train_ds = ProjectDataset.load_data_and_create_vectorizer(df.loc[df['split'] == 'train'])
  train_ds.save_vectorizer(vectorizer_path)
  vectorizer = train_ds.get_vectorizer()
  class_counts = df['nationality'].value_counts().to_dict()
  sorted_counts = sorted(class_counts.items(), key=lambda x: vectorizer.nationality_vocab.lookup_token(x[0]))
  freq = [count for _, count in sorted_counts]
  class_weights = 1.0/torch.tensor(freq, dtype=torch.float32)
  torch.save(class_weights, cw_file)

In [9]:
train_df = df.loc[df['split'] == 'train']
train_ds = ProjectDataset.load_data_and_vectorizer(train_df, vectorizer_path)
vectorizer = train_ds.get_vectorizer()
class_weights = torch.load(cw_file)
class_weights = class_weights.to(args.device)
train_dl = DataLoader(train_ds, args.batch_size, shuffle=True, drop_last=True)

In [10]:
val_df = df.loc[df['split'] == 'val']
val_ds = ProjectDataset.load_data_and_vectorizer(val_df, vectorizer_path)
val_dl = DataLoader(val_ds, args.batch_size, shuffle=True, drop_last=True)

In [11]:
test_df = df.loc[df['split'] == 'test']
test_ds = ProjectDataset.load_data_and_vectorizer(test_df, vectorizer_path)
test_dl = DataLoader(test_ds, args.batch_size, shuffle=True, drop_last=True)

In [13]:
classifier = Classifier(input_dim=len(vectorizer.surname_vocab), hidden_dim=args.hidden_dim, output_dim=len(vectorizer.nationality_vocab))
optimizer = optim.Adam(classifier.parameters(), lr=args.learning_rate)
loss_func = nn.CrossEntropyLoss(class_weights)

In [14]:
pbar = ProgressBar(persist=True)
metrics = {'accuracy': Accuracy(), 'loss': Loss(loss_func)}

In [15]:
surname_trainer = Trainer(classifier, optimizer, loss_func, train_dl, val_dl, args, pbar, metrics)
surname_trainer.run()